## 🐍 Implementación del Algoritmo Needleman-Wunsch en Python

El programa constará de tres partes principales: 
* La definición de las puntuaciones
* La función para construir la matriz
* La función para realizar el traceback.

#### Definición de puntuaciones y secuencias

Definimos las constantes según el esquema de puntuación  
* Match: +1
* Mismatch: -1
* Gap: -2


In [1]:
# Secuencias de ejemplo
secuencia_A = "GATTACA"
secuencia_B = "GCATGCU"

# Esquema de puntuación
MATCH_SCORE = 1
MISMATCH_SCORE = -1
GAP_PENALTY = -2

#### Función Principal: Construcción de la Matriz de Puntuación
Esta función crea e inicializa la matriz, y luego la rellena utilizando la ecuación de recurrencia de Needleman-Wunsch.

In [2]:
def needleman_wunsch_matrix(seq1, seq2, match, mismatch, gap):
    """Calcula y rellena la matriz de programación dinámica."""
    
    n = len(seq1) # Longitud de las filas (seq1)
    m = len(seq2) # Longitud de las columnas (seq2)
    
    # Inicializar la matriz H con (n+1) filas y (m+1) columnas
    H = [[0] * (m + 1) for _ in range(n + 1)]
    
    # 1. Inicialización de la primera fila y columna (penalización acumulada de gaps)
    for i in range(n + 1):
        H[i][0] = i * gap
    for j in range(m + 1):
        H[0][j] = j * gap
        
    # 2. Relleno de la matriz (Ecuación de Recurrencia)
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            
            # Determinar el puntaje del Match/Mismatch
            if seq1[i-1] == seq2[j-1]:
                score_diag = match
            else:
                score_diag = mismatch
            
            # Tres posibles caminos para llegar a H[i][j]:
            
            # Opción 1: Diagonal (Match/Mismatch)
            diagonal = H[i-1][j-1] + score_diag
            
            # Opción 2: Superior (Gap en seq2)
            arriba = H[i-1][j] + gap
            
            # Opción 3: Izquierda (Gap en seq1)
            izquierda = H[i][j-1] + gap
            
            # El valor de la celda es el máximo de las tres opciones
            H[i][j] = max(diagonal, arriba, izquierda)
            
    # El puntaje final es el valor de la esquina inferior derecha
    puntaje_final = H[n][m]
    
    return H, puntaje_final

#### Función de Traceback para Recuperar el Alineamiento
Una vez que la matriz H está completa, la función de traceback reconstruye el camino óptimo.

In [3]:
def traceback_alignment(H, seq1, seq2, match, mismatch, gap):
    """Realiza el traceback para encontrar el alineamiento óptimo."""
    
    n, m = len(seq1), len(seq2)
    alineacion_A = ""
    alineacion_B = ""
    
    i, j = n, m  # Comenzar en la celda final
    
    while i > 0 or j > 0:
        current_score = H[i][j]
        
        # 1. Caso Diagonal (Match/Mismatch)
        # Se verifica si la puntuación actual H[i][j] proviene de la celda diagonal superior-izquierda.
        if i > 0 and j > 0:
            if seq1[i-1] == seq2[j-1]:
                score_diag = match
            else:
                score_diag = mismatch
            
            if current_score == H[i-1][j-1] + score_diag:
                alineacion_A = seq1[i-1] + alineacion_A
                alineacion_B = seq2[j-1] + alineacion_B
                i -= 1
                j -= 1
                continue
        
        # 2. Caso Arriba (Gap en seq2)
        # Se verifica si la puntuación actual H[i][j] proviene de la celda superior (insertar gap en seq2).
        if i > 0 and current_score == H[i-1][j] + gap:
            alineacion_A = seq1[i-1] + alineacion_A
            alineacion_B = "-" + alineacion_B
            i -= 1
            continue
            
        # 3. Caso Izquierda (Gap en seq1)
        # Se verifica si la puntuación actual H[i][j] proviene de la celda izquierda (insertar gap en seq1).
        if j > 0 and current_score == H[i][j-1] + gap:
            alineacion_A = "-" + alineacion_A
            alineacion_B = seq2[j-1] + alineacion_B
            j -= 1
            continue
            
    return alineacion_A, alineacion_B

#### Ejecución y Salida (Ejemplo)
Para generar la salida solicitada

In [4]:
# --- Ejecución para la primera pareja de secuencias ---

# 1. Ejecutar Needleman-Wunsch
Matriz_completa, Puntaje_final = needleman_wunsch_matrix(
    secuencia_A, secuencia_B, MATCH_SCORE, MISMATCH_SCORE, GAP_PENALTY
)

# 2. Realizar Traceback
Alineacion1, Alineacion2 = traceback_alignment(
    Matriz_completa, secuencia_A, secuencia_B, MATCH_SCORE, MISMATCH_SCORE, GAP_PENALTY
)

# --- Generación de Salida ---
print(f"Secuencia 1: {secuencia_A}")
print(f"Secuencia 2: {secuencia_B}")
print("\n--- Alineamiento Global Óptimo ---")
print(f"Alineación A: {Alineacion1}")
print(f"Alineación B: {Alineacion2}")
print(f"\nPuntaje Final del Alineamiento: {Puntaje_final}")

print("\n--- Matriz de Puntuación Completa ---")
# Código para imprimir la matriz de manera legible
for row in Matriz_completa:
    print([f'{x:3}' for x in row])

Secuencia 1: GATTACA
Secuencia 2: GCATGCU

--- Alineamiento Global Óptimo ---
Alineación A: GATTACA
Alineación B: GCATGCU

Puntaje Final del Alineamiento: -1

--- Matriz de Puntuación Completa ---
['  0', ' -2', ' -4', ' -6', ' -8', '-10', '-12', '-14']
[' -2', '  1', ' -1', ' -3', ' -5', ' -7', ' -9', '-11']
[' -4', ' -1', '  0', '  0', ' -2', ' -4', ' -6', ' -8']
[' -6', ' -3', ' -2', ' -1', '  1', ' -1', ' -3', ' -5']
[' -8', ' -5', ' -4', ' -3', '  0', '  0', ' -2', ' -4']
['-10', ' -7', ' -6', ' -3', ' -2', ' -1', ' -1', ' -3']
['-12', ' -9', ' -6', ' -5', ' -4', ' -3', '  0', ' -2']
['-14', '-11', ' -8', ' -5', ' -6', ' -5', ' -2', ' -1']


In [5]:
def imprimir_matriz(H, seq1, seq2):
    """Función auxiliar para imprimir la matriz de puntuación con encabezados."""
    
    # Encabezado: '-' y Secuencia 2
    header = " " * 5 + ' -   ' + " ".join([f"{c:3}" for c in seq2])
    print(header)
    
    # Primera fila (inicialización)
    print(f"{'-':3} " + " ".join([f"{x:3}" for x in H[0]]))
    
    # Resto de filas
    for i in range(1, len(H)):
        row_label = seq1[i-1]
        print(f"{row_label:3} " + " ".join([f"{x:3}" for x in H[i]]))

def procesar_alineamientos(parejas):
    """
    Procesa una lista de parejas de secuencias, ejecuta NW y 
    muestra los tres resultados solicitados para cada una.
    """
    
    print("================================================================")
    print("           RESULTADOS DEL ALGORITMO NEEDLEMAN-WUNSCH            ")
    print("================================================================\n")
    
    for k, (seq_A, seq_B) in enumerate(parejas):
        
        print(f"--- PAREJA {k+1}: {seq_A} vs {seq_B} ---")
        
        # 1. Calcular Matriz y Puntaje Final
        Matriz_H, Puntaje_Final = needleman_wunsch_matrix(
            seq_A, seq_B, MATCH_SCORE, MISMATCH_SCORE, GAP_PENALTY
        )
        
        # 2. Obtener Alineamiento Óptimo mediante Traceback
        Alineacion1, Alineacion2 = traceback_alignment(
            Matriz_H, seq_A, seq_B, MATCH_SCORE, MISMATCH_SCORE, GAP_PENALTY
        )
        
        # 3. Generar la Salida Solicitada
        
        # A. La matriz de puntuación completa
        print("\n[A] Matriz de Puntuación Completa:")
        imprimir_matriz(Matriz_H, seq_A, seq_B)
        
        # B. El alineamiento global óptimo entre ambas secuencias
        print("\n[B] Alineamiento Global Óptimo:")
        print(f"Secuencia 1: {Alineacion1}")
        print(f"Secuencia 2: {Alineacion2}")
        
        # C. El puntaje final del alineamiento
        print(f"\n[C] Puntaje Final del Alineamiento: {Puntaje_Final}")
        print("----------------------------------------------------------------\n")

In [6]:
# Parejas de secuencias de nucleótidos a alinear
PAREJAS_SECUENCIAS = [
    ("GATTACA", "GCATGCU"),
    ("ACGCT", "ACGTT"),
    ("ATGCT", "AGCT"),
    ("GATTACA", "GTCGACGCA")
]

procesar_alineamientos(PAREJAS_SECUENCIAS)

           RESULTADOS DEL ALGORITMO NEEDLEMAN-WUNSCH            

--- PAREJA 1: GATTACA vs GCATGCU ---

[A] Matriz de Puntuación Completa:
      -   G   C   A   T   G   C   U  
-     0  -2  -4  -6  -8 -10 -12 -14
G    -2   1  -1  -3  -5  -7  -9 -11
A    -4  -1   0   0  -2  -4  -6  -8
T    -6  -3  -2  -1   1  -1  -3  -5
T    -8  -5  -4  -3   0   0  -2  -4
A   -10  -7  -6  -3  -2  -1  -1  -3
C   -12  -9  -6  -5  -4  -3   0  -2
A   -14 -11  -8  -5  -6  -5  -2  -1

[B] Alineamiento Global Óptimo:
Secuencia 1: GATTACA
Secuencia 2: GCATGCU

[C] Puntaje Final del Alineamiento: -1
----------------------------------------------------------------

--- PAREJA 2: ACGCT vs ACGTT ---

[A] Matriz de Puntuación Completa:
      -   A   C   G   T   T  
-     0  -2  -4  -6  -8 -10
A    -2   1  -1  -3  -5  -7
C    -4  -1   2   0  -2  -4
G    -6  -3   0   3   1  -1
C    -8  -5  -2   1   2   0
T   -10  -7  -4  -1   2   3

[B] Alineamiento Global Óptimo:
Secuencia 1: ACGCT
Secuencia 2: ACGTT

[C] Puntaje Fin

### Validación de secuencias online

https://bioboot.github.io/bimm143_W20/class-material/nw/